### Keras Tuner con el problema MNIST

Keras Tuner es una librería bastante reciente que simplifica en gran medida el ajuste de los hiperparámetros de una red neuronal. Toda la documentación en este enlace:

https://keras-team.github.io/keras-tuner/

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

Carga de los datos:

In [3]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
from time import time
import shutil

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


Normalización:

In [5]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

Lo primero que hay que hacer es definir un hipermodelo, que es una función que genera un modelo de Keras que depende de unos hiperparámetros con los que vamos a jugar. Los hiperparámetros se muestrean a partir del argumento ``hp`` de la función.

En este ejemplo sólo vamos a ajustar la constante de regularización de la capa oculta:

In [6]:
def model_builder(hp):
  hp_lambda = hp.Choice('lambda', values = [1.0, 0.1, 0.01, 0.001, 0.0001]) 
  hp_lr = hp.Choice('lr', values = [1.0, 0.1, 0.01, 0.001, 0.0001]) 
  
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units = 50, activation = 'relu', kernel_regularizer=keras.regularizers.l2(hp_lambda)))
  model.add(keras.layers.Dense(10, activation="softmax"))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_lr), 
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])    
  
  return model

Borramos el directorio de logs:

In [7]:
!rm -rf my_dir/intro_to_kt/

Lo siguiente es crear un ``tuner`` para hacer el ajuste de los hiperparámetros. Existen distintos tipos:

- RandomSearch
- Hyperband
- BayesianOptimization
- Sklearn

Lo más fácil es hacer una búsqueda aleatoria con ``RandomSearch``. Al crear el ``tuner`` hay que especificar:

- El hipermodelo.
- La variable a optimizar.
- El número total de pruebas.
- El número de ejecuciones por prueba.

In [8]:
tuner = kt.RandomSearch(model_builder, 
                        objective='val_acc',
                        max_trials=10,
                        executions_per_trial=3,
                        directory='my_dir',
                        project_name='intro_to_kt')

Un resumen del espacio de búsqueda:

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
lambda (Choice)
{'default': 1.0, 'conditions': [], 'values': [1.0, 0.1, 0.01, 0.001, 0.0001], 'ordered': True}
lr (Choice)
{'default': 1.0, 'conditions': [], 'values': [1.0, 0.1, 0.01, 0.001, 0.0001], 'ordered': True}


Y lanzamos la búsqueda:

In [10]:
tuner.search(img_train, label_train,
             epochs=1,
             validation_data=(img_test, label_test))

Trial 10 Complete [00h 00m 17s]
val_acc: 0.1092999999721845

Best val_acc So Far: 0.8940999905268351
Total elapsed time: 00h 02m 35s
INFO:tensorflow:Oracle triggered exit


Acceso al mejor modelo. Hay que tener en cuenta que ya está entrenado, y siempre es mejor reentrenarlo con todos los datos. 

In [11]:
best_model = tuner.get_best_models()[0]
best_model.evaluate(img_test, label_test)

313/313 [==============================] - 1s 1ms/step - loss: 0.7329 - acc: 0.8963


[0.732892632484436, 0.8963000178337097]

Resumen de los resultados:

In [12]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name="val_acc", direction="max")

Trial 08 summary
Hyperparameters:
lambda: 0.01
lr: 0.0001
Score: 0.8940999905268351

Trial 04 summary
Hyperparameters:
lambda: 0.1
lr: 0.001
Score: 0.878766655921936

Trial 03 summary
Hyperparameters:
lambda: 1.0
lr: 0.01
Score: 0.7130666573842367

Trial 00 summary
Hyperparameters:
lambda: 0.01
lr: 0.1
Score: 0.5470999876658121

Trial 06 summary
Hyperparameters:
lambda: 0.001
lr: 0.1
Score: 0.44673333565394086

Trial 07 summary
Hyperparameters:
lambda: 0.0001
lr: 0.1
Score: 0.4156666696071625

Trial 05 summary
Hyperparameters:
lambda: 0.001
lr: 1.0
Score: 0.11349999904632568

Trial 02 summary
Hyperparameters:
lambda: 0.0001
lr: 1.0
Score: 0.10933333386977513

Trial 09 summary
Hyperparameters:
lambda: 1.0
lr: 0.1
Score: 0.1092999999721845

Trial 01 summary
Hyperparameters:
lambda: 1.0
lr: 1.0
Score: 0.1033000002304713


Obtenemos los parámetros del mejor modelo y lo reentrenamos:

In [ ]:
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 1.2569 - acc: 0.8025 - val_loss: 0.7202 - val_acc: 0.8941
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.6427 - acc: 0.8974 - val_loss: 0.5566 - val_acc: 0.9073
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5378 - acc: 0.9071 - val_loss: 0.4915 - val_acc: 0.9161
Epoch 4/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4882 - acc: 0.9113 - val_loss: 0.4548 - val_acc: 0.9167
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4578 - acc: 0.9160 - val_loss: 0.4320 - val_acc: 0.9186
Epoch 6/10
1777/1875 [===========================>..] - ETA: 0s - loss: 0.4365 - acc: 0.9175

#### Otro ejemplo: 

Ajuste del learning rate, el parámetro de regularización y el número de neuronas en la capa oculta con un tuner de tipo hyperband.

https://arxiv.org/pdf/1603.06560.pdf

Hipermodelo:

In [ ]:
def model_builder_2(hp):
  hp_lambda = hp.Choice('lambda', values = [0.001, 0.0001]) 
  hp_units = hp.Int('units', min_value = 32, max_value = 128, step = 32)
  hp_learning_rate = hp.Choice('learning_rate', values = [1.0, 0.1, 0.01, 0.001]) 
  
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu', kernel_regularizer=keras.regularizers.l2(hp_lambda)))
  model.add(keras.layers.Dense(10, activation="softmax"))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), 
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])    
  
  return model

Borramos la carpeta de logs:

In [ ]:
!rm -rf my_dir/intro_hyperband/

Creamos el tuner:

In [ ]:
tuner = kt.Hyperband(model_builder_2,
                     objective = 'val_acc', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_hyperband')  

Resumen del espacio de búsqueda:

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
lambda (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 1.0, 'conditions': [], 'values': [1.0, 0.1, 0.01, 0.001], 'ordered': True}


Búsqueda:

In [ ]:
tuner.search(img_train, label_train,
             epochs=10,
             validation_data=(img_test, label_test))

Trial 30 Complete [00h 01m 22s]
val_acc: 0.5357999801635742

Best val_acc So Far: 0.9764999747276306
Total elapsed time: 00h 13m 49s
INFO:tensorflow:Oracle triggered exit


Mejores hiperparámetros:

In [ ]:
best_hps = tuner.get_best_hyperparameters()[0]
best_hps.values

{'lambda': 0.0001,
 'learning_rate': 0.001,
 'tuner/bracket': 2,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/round': 2,
 'tuner/trial_id': '0012',
 'units': 128}

Reentrenamiento del modelo:

In [ ]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2827 - acc: 0.9256 - val_loss: 0.1594 - val_acc: 0.9610
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1469 - acc: 0.9654 - val_loss: 0.1417 - val_acc: 0.9650
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1189 - acc: 0.9735 - val_loss: 0.1205 - val_acc: 0.9729
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1031 - acc: 0.9792 - val_loss: 0.1157 - val_acc: 0.9745
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0953 - acc: 0.9822 - val_loss: 0.1129 - val_acc: 0.9726
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0892 - acc: 0.9840 - val_loss: 0.1126 - val_acc: 0.9755
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0844 - acc: 0.9851 - val_loss: 0.1127 - val_acc: 0.9758
Epoch 8/10
1875/1875 [==============================] - 6s 3ms

### Ejercicio:

Usando el dataset Breast Cancer Wisconsin que utilizamos en el notebook ``03_log_reg_exercise.ipynb``, entrena una red neuronal para predecir el valor de la variable objetivo ``t`` optimizando los siguientes hiperparámetros:

- Número de unidades en la capa oculta. 
- Learning rate.
- Parámetro de regularización. 
- Función de activación en la capa oculta.
- Optimizador.

¿Cómo se podría optimizar el número de capas ocultas?